## Deep Learning

ΦΛΩΡΑ ΦΥΚΑ


In [1]:
import lib
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

1.  Φόρτωση του CIFAR-100

Το  dataset περιέχει 60.000 εικόνες σε 100 κατηγορίες. Κάθε εικόνα είναι σαν μονοδιάστατος πίνακας  32 x 32 pixels x 3ων  χρωμάτων RGB:
- Τα πρώτα 1024 στοιχεία είναι οι τιμές του κόκκινου. 
- Τα επόμενα 1024 στοιχεία είναι οι τιμές του πράσινου. 
- Τα τελευταία 1024 στοιχεία είναι οι τιμές του μπλε.

Δομή το αρχείου:
-	Train -> 50.000 εικόνες για εκπαίδευση
-	Test -> 10.000 εικόνες για αξιολόγηση
-	Meta -> Πληροφορίες για τις κατηγορίες 
Στο link που παρέχεται στην εκφώνηση, δίνονται οδηγίες και ο κώδικας για την ανάγνωση του αρχείου. Τα παραπάνω αρχεία δίνομαι σε μορφή Pickle και διαβάζονται σε δυαδική μορφή.


In [2]:
dataLoader = lib.DataLoader()
train_data, test_data, meta_data = dataLoader.get_data()

print(train_data.keys())  # Example: Checking available keys in train_data

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])


Διαμόρφωση δεδομένων και ετικετών

Εκτυπώνοντας τα δεδομένα των εικόνων παραπάνω, βρίσκουμε και τo ‘fine_labels’ για τις κατηγορίες των εικόνων:

In [3]:
X_train = train_data[b'data']  # Τα δεδομένα των εικόνων
y_train = np.array(train_data[b'fine_labels']) # Οι κατηγορίες των εικόνων

Με παρόμοια λογική κινούμαστε και για τα δεδομένα αξιολόγησης.

In [4]:
X_test = train_data[b'data']
y_test = np.array(train_data[b'fine_labels'])

Για τα δεδομένα των εικόνων:

In [5]:
X_train = X_train.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # (samples, height, width, channels)
X_test = X_test.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

- Με το -1 στο reshape βρίσκει αυτόματα τον αριθμό  [NumPy Documentation](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)
- Η χρήση του transpose είναι για την αναδιαμόρφωση των δεδομένων ώστε η τελική μορφή να είναι ( αριθμός εικόνων, 32, 32, 3 ) και να εξυπηρετεί καλύτερα. 

In [6]:
lib.displayDataTest(X_train, y_train, X_test, y_test)

Training data shape: (50000, 32, 32, 3)
Training labels shape: 50000
Test data shape: (50000, 32, 32, 3)
Test labels shape: 50000


2. Επιλογή 10 κλάσεων

In [7]:
label_names = dataLoader.get_names()
#print(label_names) 
selected_names = ["orange", "snake", "apple", "turtle", "bee", "bicycle", "keyboard", "pear", "plate", "telephone"]
#selected_names = lib.pick_random_classes(label_names, 10)

3.  Η εφαρμογή θα γίνει στις κλάσεις που επιλέχθηκαν τυχαία παραπάνω

Αντιστοίχιση των επιλεγμένων κλάσεων στα ονόματα των κλάσεων για να βρω τα αντίστοιχα indices, τα οποία αντιστοιχούν στις κλάσεις που θέλω να κρατήσω μέσα στα δεδομένα.

In [8]:
X_train_filtered, y_train_filtered, X_test_filtered, y_test_filtered = lib.filter_data_by_classes(
    X_train, y_train, X_test, y_test, selected_names, label_names
)

Tαξινομητής 3-NN


Οι τιμές για τις εικόνες είναι μεταξύ ( 0-255 ). Διαιρώντας με το 255 οι τιμές θα κυμένονται απο 0-1. Για καλύτερο αποτέλεσμα τα δεδομένα είναι καλύτερο να είναι σε μια κοινή κλίματα. Η κανονικοποίηση βοηθάει και στην αποφυγή κυριαρχίας μεγάλων τιμών στην απόσταση.

Επίσης για να λειτουργήσει το k-NN τα δεδομένα είναι σε μορφή 1D παράσταση, (3072,).

In [9]:
X_train_filtered, X_test_filtered = lib.normalize_and_reshape_numpy(X_train_filtered, X_test_filtered)

accuracy_knn = lib.train_and_evaluate_knn(X_train_filtered, y_train_filtered, X_test_filtered, y_test_filtered)


Accuracy: 0.70


4. Autoencoder

Ο Autoencoder είναι ένα νευρωνικό δίκτυο αποτελούμενο απο δύο μέρη:

Encoder -> θα μειώσει τη διάσταση των δεδομένων
Αποτελείται από Conv2D τα οποία συμπιέζουν την εικόνα σε διαστάσεις μικρότερες σε κάθε στρώμα. Σε κάθε επίπεδο μετατρέπει σε απλούστερες αναπαραστάσεις, πιάνοντας έτσι τα πιο σημαντικά χαρακτηριστικά της εικόνας επειδή μειώνει την πληροφορία που είναι λιγότερο σημαντική. Στο τέλος τα δεδομένα μετατρέπονται σε διάσταση 64, η οποία και είναι η συμπιεσμένη αναπαράσταση της εικόνας.

Decoder -> θα επαναφέρει τις αρχικές εικόνες
Έχουμε μια χαμηλής διάσταση αναπαράσταση των εικόνων που θα χρησιμοποιηθεί για ταξινόμηση με k-ΝΝ. 

Ο Autoencoder εκπαιδεύεται με loss function Mean Squared Error, υπολογίζοντας τη διαφορά μεταξύ της εικόνας που κατασκευάστηκε και της αρχικής εικόνας, ενώ ο αλγόριθμος Adam χρησιμοποιείται για τη βελτιστοποίηση των παραμέτρων του δικτύου. Το μοντέλο εκπαιδεύεται για 20 epochs με batch size 128.

Με τον εκπαιδευμένο autoencoder, και τον encoder εξάγουμε τις κωδικοποιημένες αναπαραστάσεις των εικόνων τις οποίες χρησιμοποιύμε ώς είσοδο για τον ταξινομητή κ-ΝΝ

Μέχρι στιγμής τα δεδομένα ήταν σε μορφή numpy representation (3072). Τώρα η μορφή των δεδομένων αλλάζει ώστε να είναι όπως χρειάζεται για το PyTorch.

In [10]:
train_loader, X_train_tensor, X_test_tensor = lib.prepare_data_for_pytorch(X_train_filtered, X_test_filtered)

autoencoder = lib.Autoencoder(encoded_dim=64)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

lib.train_autoencoder(autoencoder, train_loader, optimizer, criterion, epochs=20)

with torch.no_grad():
    X_train_encoded = autoencoder.encoder(X_train_tensor).cpu().numpy()
    X_test_encoded = autoencoder.encoder(X_test_tensor).cpu().numpy()

autoencoder_knn = lib.train_and_evaluate_knn(X_train_encoded, y_train_filtered, X_test_encoded, y_test_filtered)

Epoch 1/20, Loss: 0.0536
Epoch 2/20, Loss: 0.0585
Epoch 3/20, Loss: 0.0597
Epoch 4/20, Loss: 0.0692
Epoch 5/20, Loss: 0.0401
Epoch 6/20, Loss: 0.0314
Epoch 7/20, Loss: 0.0392
Epoch 8/20, Loss: 0.0269
Epoch 9/20, Loss: 0.0263
Epoch 10/20, Loss: 0.0249
Epoch 11/20, Loss: 0.0266
Epoch 12/20, Loss: 0.0207
Epoch 13/20, Loss: 0.0263
Epoch 14/20, Loss: 0.0177
Epoch 15/20, Loss: 0.0204
Epoch 16/20, Loss: 0.0174
Epoch 17/20, Loss: 0.0217
Epoch 18/20, Loss: 0.0213
Epoch 19/20, Loss: 0.0163
Epoch 20/20, Loss: 0.0226
Accuracy: 0.69


5. Comparizon


Δεδομένου ότι έχουμε ένα classification task για τη σύγκριση θα δοθεί περισσότερο βάση στο metric accuracy:
- είναι βασικό μέτρο αξιολόγησης για προβλήματα ταξινόμησης
- έχουμε 10 κλάσεις και θέλουμε να δούμε αν ταξινομούνται σωστά
- χρησιμοποιούμε τον ίδιο ταξινομητή k-NN σε δύο διαφορετικούς χώρους έτσι μας επιτρέπει να συγκρίνουμε αντικειμενικά raw pixels vs autoencoder embeddings
- αν ο Autoencoder πετυχαίνει υψηλότερο accuracy, σημαίνει ότι έχει μάθει καλύτερη αναπαράσταση των δεδομένων.

Θα μπορούσε να γίνει σύγκριση και με άλλα metrics όπως precision, recall, f1-score αλλά όπως φαίνεται και παρακάτω, έχουμε ένα ισορροπημένο dataset με ομοιόμορφη κατανομή στις κλάσεις, οπότε ίσως και να μην είναι τόσο χρήσιμα σε αυτή την περίπτωση.

In [11]:
train_data_by_class = lib.group_data_by_class(X_train_filtered, y_train_filtered)
test_data_by_class = lib.group_data_by_class(X_test_filtered, y_test_filtered)

for class_label, samples in train_data_by_class.items():
    print(f"Class {class_label}: {samples.shape[0]} training samples")

Class 0: 500 training samples
Class 86: 500 training samples
Class 39: 500 training samples
Class 8: 500 training samples
Class 78: 500 training samples
Class 6: 500 training samples
Class 53: 500 training samples
Class 93: 500 training samples
Class 61: 500 training samples
Class 57: 500 training samples


Autoencoder vs 3-ΝΝ ταξινομητή στον αρχικό χώρο

Στην αρχή, στα δεδομένα των εικόνων έγινε ένα reshape και μια κανονικοποίηση. Επομένως, θεωρούμε ότι βρίσκονται στον αρχικό χώρο, original feature space.

Το accuracy για το Αutencoder είναι 0.68, όπως φαίνεται σε προηγούμενο αποτέλεσμα. 

α) 3-NN 

In [12]:
accuracy_original_knn = lib.train_and_evaluate_knn(X_train_filtered, y_train_filtered, X_test_filtered, y_test_filtered)


Accuracy: 0.70


Το μοντέλο 3-NN έχει λίγο καλύτερη ακρίβεια από το Autoencoder + k-NN.
Ο Autoencoder συμπιέζει και ενδεχομένως χάνει κάποιες λεπτομέρειες που είναι σημαντικές για την ταξινόμηση.

Ωστόσο, η διαφορά 0.02 είναι πολύ μικρή, κάτι που σημαίνει ότι ο Autoencoder έχει μάθει κάποια χρήσιμα χαρακτηριστικά των δεδομένων και μπορεί να επιτύχει παρόμοια αποτελέσματα με το k-NN.

Συμπέρασμα: ο Autoencoder μπορεί να βοηθήσει στην εξαγωγή χρήσιμων χαρακτηριστικών αλλά δεν είναι πάντα καλύτερο απο το απλό k-NN.  

β) CLIP

Στο paper "Learning Transferable Visual Models From Natural Language Supervision" υπάρχει link για το επίσημο repository για το CLIP. Ακολουθώντας τις οδηγίες εγκατέστησα τα απαραίτητα πακέτα εκτός από το cuda toolkit καθώς επέλεξα να χρησιμοποιήσω τη cpu αντί τη gpu. 

Πολύ ενδιαφέρον έχει ότι το CLIP είναι ένα προεκπαιδευμένο μοντέλο που αντιστοιχίζει εικόνες με κάποια περιγραφή, όπως λέξη, ετικέτα κλπ σε φυσική γλώσσα όπως στη δική μας περίπτωση και ταξινομεί χωρίς επιπλέον εκπαίδευση πάνω σε συγκεκριμένες κατηγορίες, zero-shot classification όπως αναφέρεται και στο paper.

Στη δική μου υλοποίηση:

- Χρησιμοποίησα το μοντέλο ViT-B/32 από το clip repository
- Για τα test data έκανα ένα reshape σε ( samples, height, width, channels ) όπως τα χρειάζεται το μοντέλο CLIP.
- Για κάθε όνομα των κατηγοριών που επέλεξα έφτιαξα με το encode_text προτάσεις με τη συγκεκριμένη μορφή που το CLIP χρησιμοποιεί. Προτάσεις όπως "a photo of a X".
- Η επιλογή κατηγορίας γίνεται με βάση το εσωτερικό γινόμενο μεταξύ εικόνας και text embeddings. Γίνεται δηλαδή δοκιμή κάθε εικόνας με τις περιγραφές και κρατάμε αυτή που της ταιριάζει περισσότερο. 
- Τέλος γίνεται υπολογισμός τησ ακρίβειας πάνω στο test set.

In [13]:
y_test_filtered_names = [label_names[i] for i in y_test_filtered]
acc_clip = lib.zero_shot_clip_predict(X_test_filtered, y_test_filtered_names, selected_names)
print(f"Zero-Shot CLIP Accuracy: {acc_clip:.4f}")

CLIP Zero-Shot Predict: 100%|██████████| 5000/5000 [19:48<00:00,  4.21it/s]


Zero-Shot CLIP Accuracy: 0.9014


Το μοντέλο CLIP πεετυχαίνει καλύτερα αποτελέσματα από τα k-NN και Autoencoder + k-NN. H ακρίβεια του φτάνει το 90%. Σε σύγκριση με περίπου 70% που έχει το μοντέλο 3-NN είναι περίπου 20% καλύτερο στην ταξινόμηση των εικόνων στις 10 επιλεγμένες κατηγορίες.

Το πιο εντυπωσιακό είναι ότι το μοντέλο δεν έχει εκπαιδευτεί πάνω σε αυτό το dataset, ούτε έγινε κάποια προσαρμογή για τη δική μου χρήση. Το μόνο που έκανα ήταν να του δώσω κάποιες προτάσεις ως ετικέτες και τις εικόνες.

Συμπέρασμα: Το μοντέλο CLIP είναι πολύ καλό στην αναγνώριση εικόνων, τουλάχιστον με βάση τις εικόνες που του έδωσα στο δικό μου δείγμα.

γ) Αλλο εκπαιδευμένο embedding (off-the-shelf):  ResNet-50

Για το embedding που επέλεξα, χρησιμοποίησα το μοντέλο ResNet-50 επειδή έχει εκπαιδευτεί στο dataset ImageNet. Αυτό το μοντέλο θα έπρεπε να είναι κατάλληλο για ταξινόμηση τύπου zero-shot, αλλά ενδέχεται να μην είναι τόσο ακριβές όσο το μοντέλο CLIP, τουλάχιστον όχι χωρίς fine-tuning. 

Στη δική μου υλοποίηση:

- Χρησιμοποίησα την προεκπαιδευμένη έκδοση που παρέχεται από το TorchVision.
- Για τα test data έκανα ένα reshape σε (samples, height, width, channels) όπως χρειάζεται το μοντέλο ResNet-50. 
- Έκανα τις απαραίτητες μετατροπές προεπεξεργασίας ώστε η εικόνα να είναι στη μορφή που χρειάζεται το ResNet-50. 
- Αυτή τη φορά δεν χρειάζονται prompts, όπως έγινε με το μοντέλο CLIP. Παίρνουμε την πρόβλεψη από το ResNet και τη χρησιμοποιούμε ως δείκτη για να εντοπίσουμε την αντίστοιχη κατηγορία.
- Τέλος γίνεται υπολογισμός της ακρίβειας πάνω στο test set.

In [14]:
acc_res = lib.resnet_predict(X_test_filtered, y_test_filtered_names, selected_names)
print(f"Zero-Shot ResNet50 Accuracy: {acc_res:.4f}")

ResNet-50 Prediction: 100%|██████████| 5000/5000 [07:05<00:00, 11.75it/s]

Zero-Shot ResNet50 Accuracy: 0.1000


Το μοντέλο ResNet-50 δε φαίνεται να πετυχαίνει το επιθυμητό αποτέλεσμα όταν χρησιμοποιείται ως προεκπαιδευμένο. Έχει μόνο 10% ακρίβεια, κάτι που το κάνει το μοντέλο μ ετη χαμηλότερη απόδοση, από αυτά που συγκρίθηκαν.

Δεν είναι σαφές γιατί συμβαίνει αυτό, καθώς η υλοποίηση ακολουθεί το ίδιο μοτίβο με αυτό του μοντέλου CLIP. Ένας πιθανός λόγος θα μπορούσε να είναι ότι το dataset στο οποίο εκπαιδεύτηκε το ResNet-50, δηλαδή το ImageNet, ίσως δεν έχει τόση ποικιλία στις ετικέτες όσο θα περιμέναμε για το dataset που χρησιμοποιήθηκε σε αυτή την άσκηση.

Επιπλέον, το μοντέλο δεν έγινε fine-tuned, κάτι που μπορεί να επηρέασε τα αποτελέσματα.

Συμπέρασμα: Το ResNet-50 τουλάχιστον όπως εφαρμόστηκε εδώ, δεν φαίνεται να είναι η κατάλληλη επιλογή για μοντέλο ταξινόμησης εικόνων τύπου zero-shot στο dataset CIFAR-100.

6. Επιλογή τυχαίων κλάσεων και τελική σύγκριση

Η παραπάνω ανάλυση έγινε έχοντας επιλέξει συγκεκριμένες κλάσεις. Αν και η επιλογή ήταν εν μέρει τυχαία, είχα στόχο να διευκολύνω τα μοντέλα επιλέγοντας οντότητες με έντονες διαφορές πχ. δεν συνδύασα παρόμοιες κατηγορίες όπως σκύλος-λύκος, αλλά αντίθετες όπως φίδι-πορτοκάλι.

Για να μια πιο γενική εικόνα της απόδοσης, επανέλαβα την ίδια ανάλυση με πλήρως τυχαία δειγματοληψία από τις κλάσεις του dataset.
Παρακάτω είναι τα αποτελέσματα: 

In [15]:
#lib.print_model_performance(selected_names, accuracy_knn, autoencoder_knn, accuracy_original_knn, acc_clip, acc_res)

### Επιλεγμένες κλάσεις
`orange`, `snake`, `apple`, `turtle`, `bee`, `bicycle`, `keyboard`, `pear`, `plate`, `telephone`

| Model              | Accuracy |
|--------------------|----------|
| KNN                | 0.6990   |
| Autoencoder KNN    | 0.6768   |
| KNN (Original)     | 0.6990   |
| CLIP               | 0.9008   |
| ResNet-50          | 0.1308   |


### Τυχαία Επιλεγμένες κλάσεις

1: `baby`, `bear`, `beaver`, `mountain`, `pine_tree`, `beetle`, `skyscraper`, `chimpanzee`, `wolf`, `sea`

| Model              | Accuracy |
|--------------------|----------|
| KNN                | 0.6562   |
| Autoencoder KNN    | 0.6726   |
| KNN (Original)     | 0.6562   |
| CLIP               | 0.8714   |
| ResNet-50          | 0.0892   |


2: `ray`, `mouse`, `hamster`, `beetle`, `cup`, `telephone`, `trout`, `rocket`, `palm_tree`, `chair`

| Model              | Accuracy |
|--------------------|----------|
| KNN                | 0.7000   |
| Autoencoder KNN    | 0.6368   |
| KNN (Original)     | 0.7000   |
| CLIP               | 0.8556   |
| ResNet-50          | 0.0434   |


3: `mouse`, `trout`, `chimpanzee`, `chair`, `motorcycle`, `spider`, `willow_tree`, `castle`, `streetcar`, `worm`

| Model              | Accuracy |
|--------------------|----------|
| KNN                | 0.6770   |
| Autoencoder KNN    | 0.6712   |
| KNN (Original)     | 0.6770   |
| CLIP               | 0.9148   |
| ResNet-50          | 0.0856   |

Ανεξάρτητα από το ποιές κλάσεις επιλέχθηκαν, το CLIP αποδίδει σταθερά καλύτερα έναντι των άλλων μοντέλων. Διατηρεί πολυ υψηλό accuracy πάνω απο 85% ακόμα και με τυχαία δειγματοληψία. 

Τα παραδοσιακά μοντέλα k-NN και Αutoencoder + k-NN, έχουν σταθερό αλλά χαμηλό accuracy, περίπου 65-70%. Αξιοπρόσεκτο είναι ότι ο Autoencoder δε φαίνεται να βελτιώνει σημαντικά την απόδοση του απλού k-NN, σε κάποιες περιπτώσεις είναι ελαφρώς χειρότερο. 

Το ResNet-50 είχε πολύ χαμηλή ακρίβεια, που μάλλον σημαίνει ότι δεν έχει προσαρμοστεί καλά στο συγκεκριμένο σύνολο δεδομένων ή απλώς δε λειτουργεί καλά με τον τρόπο που χρησιμοποιήθηκε εδώ.